# Dataset Exploration Overview 
Comprehensive analysis of retail inventory forecasting dataset

## Importing pandas to load Retail Sales .csv file and explore data

In [108]:
import pandas as pd

## Configuring display properties

In [109]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Loading dataset into data frame for examination

In [110]:
df = pd.read_csv('../data/raw/retail_store_inventory.csv')

## Key analysis points

In [111]:
print(f"Dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage().sum()/1024**2:.2f} MB")
print(f"Column types:\n{df.dtypes}\n\n")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Number of stores: {df['Store ID'].nunique()}")
print(f"Number of products: {df['Product ID'].nunique()}")
#print(f"Number of Store-Product-Date combinations: {df[['Store ID', 'Product ID', 'Date']].nunique()}")
print(f"Product categories: {df['Category'].unique()}")
print(f"Regions: {df['Region'].unique()}")
print(f"Weather conditions: {df['Weather Condition'].unique()}")
print(f"Seasons: {df['Seasonality'].unique()}")

Dataset shape: (73100, 15)
Memory usage: 8.37 MB
Column types:
Date                   object
Store ID               object
Product ID             object
Category               object
Region                 object
Inventory Level         int64
Units Sold              int64
Units Ordered           int64
Demand Forecast       float64
Price                 float64
Discount                int64
Weather Condition      object
Holiday/Promotion       int64
Competitor Pricing    float64
Seasonality            object
dtype: object


Date range: 2022-01-01 to 2024-01-01
Number of stores: 5
Number of products: 20
Product categories: ['Groceries' 'Toys' 'Electronics' 'Furniture' 'Clothing']
Regions: ['North' 'South' 'West' 'East']
Weather conditions: ['Rainy' 'Sunny' 'Cloudy' 'Snowy']
Seasons: ['Autumn' 'Summer' 'Winter' 'Spring']


## Data quality

In [116]:
print(f"\nNumber of missing values: {df.isnull().sum().sum()}")
print(f"Number of duplicated values: {df.duplicated().sum()}")

df['Date'] = pd.to_datetime(df['Date'])
df_sorted = df.sort_values('Date')
date_diff = df_sorted['Date'].diff()
gaps = date_diff[date_diff > pd.Timedelta(days=1)]

if len(gaps) > 0:
    print(f"Date gaps found: {len(gaps)} instances")
    print("Largest gaps:")
    print(gaps.sort_values(ascending=False).head())
else:
    print("No date gaps detected - continuous daily data")


Number of missing values: 0
Number of duplicated values: 0
No date gaps detected - continuous daily data


## Sales analysis by day-of-the-week

In [132]:
df['Day of Week'] = df['Date'].dt.day_name()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

print(f"Unique weeks in dataset: {df['Date'].dt.to_period('W').nunique()}")
print("\nRecords per day of week:")
# Reorder by actual day of week
print(df['Day of Week'].value_counts().reindex(day_order))

print("\nTotal and average sales by day of week:")
dow_sales = df.groupby('Day of Week')['Units Sold'].agg(['sum', 'mean']).round(2)
# Reorder by actual day of week
dow_sales = dow_sales.reindex(day_order)
print(dow_sales)

Unique weeks in dataset: 106

Records per day of week:
Day of Week
Monday       10500
Tuesday      10400
Wednesday    10400
Thursday     10400
Friday       10400
Saturday     10500
Sunday       10500
Name: count, dtype: int64

Total and average sales by day of week:
                 sum    mean
Day of Week                 
Monday       1418494  135.09
Tuesday      1428000  137.31
Wednesday    1420797  136.62
Thursday     1427732  137.28
Friday       1423255  136.85
Saturday     1420802  135.31
Sunday       1436502  136.81


## Sales analysis by month

In [133]:
df['Month'] = df['Date'].dt.month_name()
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

print(f"Unique months in dataset: {df['Date'].dt.to_period('M').nunique()}")

print("\nRecords per month:")
# Reorder by actual month order
print(df['Month'].value_counts().reindex(month_order))

print("\nTotal and average sales by month:")  
month_sales = df.groupby('Month')['Units Sold'].agg(['sum', 'mean']).round(2)
# Reorder by actual month order
month_sales = month_sales.reindex(month_order)
print(month_sales)


Unique months in dataset: 25

Records per month:
Month
January      6300
February     5600
March        6200
April        6000
May          6200
June         6000
July         6200
August       6200
September    6000
October      6200
November     6000
December     6200
Name: count, dtype: int64

Total and average sales by month:
              sum    mean
Month                    
January    856590  135.97
February   776220  138.61
March      842595  135.90
April      808457  134.74
May        833472  134.43
June       821150  136.86
July       864547  139.44
August     841417  135.71
September  816971  136.16
October    852509  137.50
November   830661  138.44
December   830993  134.03


## External Factor Analysis

In [134]:

print("Weather condition occurrence count:")
print(df['Weather Condition'].value_counts())

print("\nAverage sales by weather condition:")
weather_impact = df.groupby('Weather Condition')['Units Sold'].agg(['mean', 'std']).round(2)
print(weather_impact)


holiday_col = [col for col in df.columns if 'holiday' in col.lower() or 'promotion' in col.lower()]
if holiday_col:
    col_name = holiday_col[0]
    print(f"\n{col_name} impact:")
    print(df.groupby(col_name)['Units Sold'].agg(['count', 'mean', 'std']).round(2))


Weather condition occurrence count:
Weather Condition
Sunny     18290
Rainy     18278
Snowy     18272
Cloudy    18260
Name: count, dtype: int64

Average sales by weather condition:
                     mean     std
Weather Condition                
Cloudy             136.76  108.33
Rainy              135.16  107.92
Snowy              135.91  109.38
Sunny              138.03  110.02

Holiday/Promotion impact:
                   count    mean     std
Holiday/Promotion                       
0                  36747  136.51  109.28
1                  36353  136.42  108.56


## First 5 rows of dataset

In [135]:
df.head(5)

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,Day_of_Week,Month,Day of Week
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,Saturday,January,Saturday
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn,Saturday,January,Saturday
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer,Saturday,January,Saturday
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn,Saturday,January,Saturday
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer,Saturday,January,Saturday
